In [1]:
import pandas as pd
import requests, json
from pprint import pprint

pd.set_option('display.max_columns', None)

In [2]:
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

players = r['elements']

# show data for first player
pprint(players[0])

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 2843780}
{'assists': 3,
 'bonus': 0,
 'bps': 67,
 'chance_of_playing_next_round': None,
 'chance_of_playing_this_round': None,
 'clean_sheets': 1,
 'clean_sheets_per_90': 0.31,
 'code': 438098,
 'corners_and_indirect_freekicks_order': None,
 'corners_and_indirect_freekicks_text': '',
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': 0,
 'cost_change_start_fall': 0,
 'creativity': '88.8',
 'creativity_rank': 267,
 'creativity_rank_type': 146,
 'direct_freekicks_order': None,
 'direct_freekicks_text': '',
 'dreamteam_count': 0,
 'element_type': 3,
 'ep_next': '3.0',
 'ep_this': None,
 'event_points': 0,
 'expected_assists': '0.64',
 'expected_assists_per_90': 0.2,
 'expected_goal_involvements': '1.89',
 'expected_goal_involvements_per_90': 0.59,
 'expected_goals': '1.25',
 'expected_goals_c

In [3]:
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Fábio Vieira,1,3
1,2,G.Jesus,1,4
2,3,Gabriel,1,2
3,4,Havertz,1,4
4,5,Hein,1,1


In [4]:
teams = pd.json_normalize(r['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,5,None,False,0,1350,1380,1370,1370,1330,1390,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1120,1245,1110,1140,1130,1350,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1100,1100,1075,1100,1130,1105,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1100,1100,1105,1095,1100,1110,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1100,1100,1100,1105,1100,1100,131


In [5]:
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],63
1,2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],193
2,3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],255
3,4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],68


In [6]:
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# show joined result
df[['first_name', 'second_name', 'name']].head()

,first_name,second_name,name
0,Fábio,Ferreira Vieira,Arsenal
1,Gabriel,Fernando de Jesus,Arsenal
2,Gabriel,dos Santos Magalhães,Arsenal
3,Kai,Havertz,Arsenal
4,Karl,Hein,Arsenal


In [7]:
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
    ['first_name', 'second_name', 'team_name', 'position_name']
].head()

,first_name,second_name,team_name,position_name
0,Fábio,Ferreira Vieira,Arsenal,Midfielder
1,Jorge Luiz,Frello Filho,Arsenal,Midfielder
2,Gabriel,Martinelli Silva,Arsenal,Midfielder
3,Reiss,Nelson,Arsenal,Midfielder
4,Ethan,Nwaneri,Arsenal,Midfielder


In [11]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df


# show player #1's gameweek history
get_season_history(1)[
    [
        'season_name',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)

,season_name,total_points,minutes,goals_scored,assists
0,2022/23,40,500,1,2
1,2023/24,24,290,1,3


In [12]:
get_season_history(2)[
    [
        'season_name',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)

,season_name,total_points,minutes,goals_scored,assists
0,2016/17,67,651,7,4
1,2017/18,126,1660,13,7
2,2018/19,79,1017,7,3
3,2019/20,146,2018,14,8
4,2020/21,115,2056,9,5
5,2021/22,120,1871,8,8
6,2022/23,125,2064,11,7
7,2023/24,85,1470,4,7


In [13]:
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

,id_player,first_name,second_name,web_name,name,singular_name_short
0,1,Fábio,Ferreira Vieira,Fábio Vieira,Arsenal,MID
1,7,Jorge Luiz,Frello Filho,Jorginho,Arsenal,MID
2,9,Gabriel,Martinelli Silva,Martinelli,Arsenal,MID
3,10,Reiss,Nelson,Nelson,Arsenal,MID
4,12,Ethan,Nwaneri,Nwaneri,Arsenal,MID


In [14]:
from tqdm.auto import tqdm
tqdm.pandas()

In [16]:
points = players['id_player'].progress_apply(get_season_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

  0%|          | 0/579 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
points.groupby(
    ['element', 'web_name']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head()

In [20]:
base_url = 'https://fantasy.premierleague.com/api/'

def fetch_fpl_data(url):
    response = requests.get(url)
    response.raise_for_status()  # Upewnij się, że żądanie zakończyło się sukcesem
    return response.json()
# Funkcja do pobierania danych za poszczególne kolejki
def get_gameweek_data(season, gameweek):
    url = f'{base_url}event/{gameweek}/live/'
    data = fetch_fpl_data(url)
    return data

# Przykład: Pobieranie danych dla konkretnej kolejki w sezonie 2020/21
season = '2023-24'  # Ustaw sezon, który chcesz pobrać
gameweek = 1       # Ustaw numer kolejki

data = get_gameweek_data(season, gameweek)

In [22]:
base_url = 'https://fantasy.premierleague.com/api/'

# Pobieranie podstawowych danych
bootstrap_static_url = f'{base_url}bootstrap-static/'
bootstrap_data = fetch_fpl_data(bootstrap_static_url)

# Przykładowo, możemy sprawdzić dane zawodników
players = bootstrap_data['elements']
players_df = pd.DataFrame(players)


In [23]:
players_df

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,NaN,None,438098,0,0,0,0,0,3,3.0,None,0,Fábio,0.0,1,False,,None,55,438098.jpg,2.2,Ferreira Vieira,0.1,False,None,a,1,3,24,0,0,0,0,0.0,4.4,Fábio Vieira,290,1,3,1,2,0,0,0,0,1,0,0,67,82.8,88.8,85.0,25.5,2,1.25,0.64,1.89,1.83,335,146,267,146,253,130,321,147,NaN,,NaN,,4.0,,0.39,0.0,0.20,0.59,0.57,0.62,141,86,468,188,250,104,453,152,0.62,0.31
1,NaN,None,205651,0,0,0,0,0,4,3.2,None,0,Gabriel,0.0,2,False,,None,70,205651.jpg,3.1,Fernando de Jesus,1.4,False,None,a,1,3,85,0,0,0,0,0.0,12.1,G.Jesus,1470,4,7,11,11,0,0,0,6,0,0,10,271,364.8,403.0,756.0,152.7,17,6.27,3.26,9.53,11.67,184,20,84,10,30,16,71,16,NaN,,NaN,,NaN,,0.38,0.0,0.20,0.58,0.71,0.67,29,12,549,54,143,22,154,27,1.04,0.67
2,NaN,None,226597,0,0,0,0,0,2,4.2,None,0,Gabriel,0.0,3,False,,None,60,226597.jpg,4.1,dos Santos Magalhães,12.0,False,None,a,1,3,149,0,0,0,0,0.0,24.8,Gabriel,3042,3,1,16,26,1,0,0,4,0,0,8,676,675.8,121.9,379.0,117.9,34,4.48,0.65,5.13,26.18,59,18,238,73,91,4,110,21,NaN,,NaN,,NaN,,0.13,0.0,0.02,0.15,0.77,0.77,81,6,503,188,42,6,31,12,1.01,0.47
3,NaN,None,219847,0,0,0,0,0,4,3.5,None,0,Kai,0.0,4,False,,None,80,219847.jpg,4.9,Havertz,13.6,False,None,a,1,3,180,0,0,0,0,0.0,22.5,Havertz,2627,13,10,16,22,0,0,0,11,0,0,15,515,763.2,613.4,1027.0,240.2,30,12.36,3.88,16.23,21.03,43,8,47,2,14,7,19,5,NaN,,NaN,,3.0,,0.42,0.0,0.13,0.55,0.72,0.75,12,4,568,65,18,4,27,5,1.03,0.55
4,NaN,None,463748,0,0,0,0,0,1,2.0,None,0,Karl,0.0,5,False,,None,40,463748.jpg,0.0,Hein,0.2,False,None,a,1,3,0,0,0,0,0,0.0,0.0,Hein,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,443,52,434,40,408,24,446,52,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,558,60,55,23,445,52,354,53,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,NaN,None,483081,0,0,0,0,0,3,1.0,None,0,Rodrigo,0.0,563,False,,None,55,483081.jpg,0.0,Martins Gomes,0.2,False,None,a,20,39,0,0,0,0,0,0.0,0.0,R.Gomes,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,570,250,570,250,570,250,570,250,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,155,93,482,195,570,250,408,134,0.00,0.00
575,NaN,None,231480,0,0,0,0,0,2,0.9,None,0,Santiago,0.0,564,False,,None,45,231480.jpg,1.1,Bueno,0.2,False,None,a,20,39,13,0,0,0,0,0.0,2.9,S.Bueno,819,0,0,0,18,0,0,0,1,0,0,0,124,173.8,27.1,2.0,20.3,9,0.00,0.01,0.01,20.71,289,106,326,120,356,131,331,116,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,2.28,1.98,406,110,175,119,353,122,43

In [25]:
def fetch_fpl_data(url):
    response = requests.get(url)
    response.raise_for_status()  # Upewnij się, że żądanie zakończyło się sukcesem
    return response.json()

# Podstawowy URL API FPL
base_url = 'https://fantasy.premierleague.com/api/'

# Funkcja do pobierania danych za poszczególne kolejki
def get_gameweek_data(gameweek):
    url = f'{base_url}event/{gameweek}/live/'
    data = fetch_fpl_data(url)
    return data

# Przykład: Pobieranie danych dla konkretnej kolejki
gameweek = 1  # Ustaw numer kolejki
data = get_gameweek_data(gameweek)

if 'elements' in data:
    elements = data['elements']
    df = pd.DataFrame(elements)
    print(df)
else:
    print('No data available for this gameweek.')

Empty DataFrame
Columns: []
Index: []


In [26]:
def fetch_fpl_data(url):
    response = requests.get(url)
    response.raise_for_status()  # Upewnij się, że żądanie zakończyło się sukcesem
    return response.json()

# Podstawowy URL API FPL
base_url = 'https://fantasy.premierleague.com/api/'

# Pobieranie danych za cały sezon
def get_season_data():
    url = f'{base_url}bootstrap-static/'
    data = fetch_fpl_data(url)
    return data

# Przykład: Pobieranie danych za sezon 2020/21
season_data = get_season_data()

# Sprawdzenie dostępnych kluczy w danych
print(season_data.keys())

# Konwersja danych do DataFrame i zapisanie do pliku CSV
players = season_data['elements']
players_df = pd.DataFrame(players)
players_df

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])


,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,NaN,None,438098,0,0,0,0,0,3,3.0,None,0,Fábio,0.0,1,False,,None,55,438098.jpg,2.2,Ferreira Vieira,0.1,False,None,a,1,3,24,0,0,0,0,0.0,4.4,Fábio Vieira,290,1,3,1,2,0,0,0,0,1,0,0,67,82.8,88.8,85.0,25.5,2,1.25,0.64,1.89,1.83,335,146,267,146,253,130,321,147,NaN,,NaN,,4.0,,0.39,0.0,0.20,0.59,0.57,0.62,141,86,468,188,250,104,453,152,0.62,0.31
1,NaN,None,205651,0,0,0,0,0,4,3.2,None,0,Gabriel,0.0,2,False,,None,70,205651.jpg,3.1,Fernando de Jesus,1.4,False,None,a,1,3,85,0,0,0,0,0.0,12.1,G.Jesus,1470,4,7,11,11,0,0,0,6,0,0,10,271,364.8,403.0,756.0,152.7,17,6.27,3.26,9.53,11.67,184,20,84,10,30,16,71,16,NaN,,NaN,,NaN,,0.38,0.0,0.20,0.58,0.71,0.67,29,12,549,54,143,22,154,27,1.04,0.67
2,NaN,None,226597,0,0,0,0,0,2,4.2,None,0,Gabriel,0.0,3,False,,None,60,226597.jpg,4.1,dos Santos Magalhães,12.0,False,None,a,1,3,149,0,0,0,0,0.0,24.8,Gabriel,3042,3,1,16,26,1,0,0,4,0,0,8,676,675.8,121.9,379.0,117.9,34,4.48,0.65,5.13,26.18,59,18,238,73,91,4,110,21,NaN,,NaN,,NaN,,0.13,0.0,0.02,0.15,0.77,0.77,81,6,503,188,42,6,31,12,1.01,0.47
3,NaN,None,219847,0,0,0,0,0,4,3.5,None,0,Kai,0.0,4,False,,None,80,219847.jpg,4.9,Havertz,13.6,False,None,a,1,3,180,0,0,0,0,0.0,22.5,Havertz,2627,13,10,16,22,0,0,0,11,0,0,15,515,763.2,613.4,1027.0,240.2,30,12.36,3.88,16.23,21.03,43,8,47,2,14,7,19,5,NaN,,NaN,,3.0,,0.42,0.0,0.13,0.55,0.72,0.75,12,4,568,65,18,4,27,5,1.03,0.55
4,NaN,None,463748,0,0,0,0,0,1,2.0,None,0,Karl,0.0,5,False,,None,40,463748.jpg,0.0,Hein,0.2,False,None,a,1,3,0,0,0,0,0,0.0,0.0,Hein,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,443,52,434,40,408,24,446,52,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,558,60,55,23,445,52,354,53,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,NaN,None,483081,0,0,0,0,0,3,1.0,None,0,Rodrigo,0.0,563,False,,None,55,483081.jpg,0.0,Martins Gomes,0.2,False,None,a,20,39,0,0,0,0,0,0.0,0.0,R.Gomes,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,570,250,570,250,570,250,570,250,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,155,93,482,195,570,250,408,134,0.00,0.00
575,NaN,None,231480,0,0,0,0,0,2,0.9,None,0,Santiago,0.0,564,False,,None,45,231480.jpg,1.1,Bueno,0.2,False,None,a,20,39,13,0,0,0,0,0.0,2.9,S.Bueno,819,0,0,0,18,0,0,0,1,0,0,0,124,173.8,27.1,2.0,20.3,9,0.00,0.01,0.01,20.71,289,106,326,120,356,131,331,116,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,2.28,1.98,406,110,175,119,353,122,43

In [27]:
url = 'https://fantasy.premierleague.com/api/fixtures/'
data = fetch_fpl_data(url)
data

[{'code': 2444470,
  'event': 1,
  'finished': False,
  'finished_provisional': False,
  'id': 1,
  'kickoff_time': '2024-08-16T19:00:00Z',
  'minutes': 0,
  'provisional_start_time': False,
  'started': False,
  'team_a': 9,
  'team_a_score': None,
  'team_h': 14,
  'team_h_score': None,
  'stats': [],
  'team_h_difficulty': 2,
  'team_a_difficulty': 3,
  'pulse_id': 115827},
 {'code': 2444473,
  'event': 1,
  'finished': False,
  'finished_provisional': False,
  'id': 4,
  'kickoff_time': '2024-08-17T11:30:00Z',
  'minutes': 0,
  'provisional_start_time': False,
  'started': False,
  'team_a': 12,
  'team_a_score': None,
  'team_h': 10,
  'team_h_score': None,
  'stats': [],
  'team_h_difficulty': 4,
  'team_a_difficulty': 2,
  'pulse_id': 115830},
 {'code': 2444471,
  'event': 1,
  'finished': False,
  'finished_provisional': False,
  'id': 2,
  'kickoff_time': '2024-08-17T14:00:00Z',
  'minutes': 0,
  'provisional_start_time': False,
  'started': False,
  'team_a': 20,
  'team_a_sc